In [ ]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:

from datasets import load_dataset 
dataset = load_dataset("Ahmed1902/upgrade_bassoom")
dataset


In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
device=f'{cuda.current_device()}' if cuda.is_available() else 'cpu'


In [ ]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,

).to(device)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
system_message="""You are a psychotherapy assistant responsible for helping and guiding individuals with questions about mental health issues. Your role involves actively listening to users, understanding their needs, and providing responses that align with their questions to ensure they feel understood. Your primary goal is to encourage them to seek professional help and offer useful ways to contact specialists."""
def format_llama(entry):
  prompt = f"<s>[INST] <<SYS>>{system_message}<</SYS>>{entry['Questions']} [/INST]  {entry['Answers']}  </s>"

  return prompt

In [ ]:
dataset['train'][0]

In [ ]:

from datasets import load_dataset
from sklearn.model_selection import train_test_split

train_data = dataset['train']

# Split the dataset into training and validation sets
train, val = train_test_split(train_data, test_size=0.2, random_state=42)



In [ ]:
trainer = SFTTrainer(
    model=model_id,
    train_dataset=train_data,
    peft_config=peft_params,
    packing=True,
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    formatting_func=format_llama,
    
)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:

trainer.train()
trainer.save_model()